# Chapter 8. Dimensionality Reduction
___

: 대부분의 Machine Learning 문제들은 각각의 training instance 마다 매우 많은 특성들(features)을 가지고 있다.

이와같은 데이터들은 training의 속도를 매우 늦출뿐 아니라, 좋은 solution을 찾는 것 또한 더욱 어렵게 만든다.

이러한 형상은 '***curse of dimensionality***' 라고 언급된다.

다행히도 실제 문제에서는 dimension의 수를 상당히 줄이는 것이 종종 가능하여 난해한 문제를 다루기 쉬운 문제로 전환시키기도 한다. (e.g. MNIST images)

<***Warning***>

> Dimensionality를 줄이는 것은 몇몇의 정보를 잃는 것과 같다. 그러므로 이는 training의 속도를 올려주지만 약간 system 의 기능을 낮춘다. </br>
> 그리고 이는 piplines을 약간 더 복잡하게 만들고 유지하기 어렵게 만든다. </ㅠㄱ>
> **그러므로 training이 너무 느려서 *Dimensionality Reduction*을 고려해보기 이전에, *먼저 original data를 통해 system을 학습*시켜보아야 한다.** </br>
> 몇몇의 경우에서는 training data의 dimensionality를 줄이는 것은 noise나 불필요한 정보들을 걸러줄 수 도있고, 더 좋은 결과를 보이기도 한다. </br>
> 하지만, **보통의 경우에서는 좋은 결과를 내지는 않는다. 그저 속도만 높여준다.**

: training을 더 빠르게 할 수 있는다는 것과는 별개로, **dimensionality reduction**은 ***data visualization (DataViz)***에 아주 유용하다.

    - dimension들의 수를 2~3정도로 줄이는 것은 고차원 training set을 graph로 plot 할 수 있고, cluster와 같은 pattern을 시각적으로 감지하여 몇 가지 중요한 관점을 얻는 경우가 많다.
    
이번 장에서는 데이터의 차원을 축소하는 방법인 투영(projection) 과 manifold learning 그리고 대표적인 dimensionality algorithm인 ' ***주 성분 분석 (PCA), Kernal PCA, LLE*** ' 등에 대해 알아볼 것이다.

</br>

___

## 8.1 The Curse of Dimensionality

머신러닝에서 데이터 셋의 특성(feature)가 많아지면, 각 특성인 하나의 차원(dimension) 또한 증가하게 된다. 

이렇게 데이터의 차원이 증가할 수록 데이터 공간의 부피가 기하 급수적으로 증가하기 때문에, 데이터의 밀도는 차원이 증가할 수록 희소(sparse)해진다. 

아래의 그림은 데이터의 차원이 증가할수록 각 영역(region)에 포함되는 데이터의 개수를 나타낸 그림(출처: jermwatt's blog)이다. 

그림에서 알 수 있듯이 차원이 증가할 수록 데이터의 밀도가 희소해지는것을 알 수 있다.

![img_1](./imgs/8-1-1.png)

데이터의 차원이 증가할수록 데이터 포인트 간의 거리 또한 증가하게 되므로, 이러한 데이터를 이용해 머신러닝 알고리즘을 학습 하게되면 모델이 복잡해지게 된다. 

따라서, **오버피팅(overfitting)** 위험이 커진다. 

이러한 curse of dimensionality를 해결하기 위한 방법 중 하나는 데이터의 밀도가 높아질 때까지 train dataset을 늘리는 것이다. 

하지만, dataset의 크기에 비해 dimension은 기하급수적으로 커지기 때문에 매우 힘든일이라 할 수 있다.


</br>

___

## 8.2 Main Approaches for Dimensionality Reduction

: Dimensionality Reduction의 두 가지 접근 방법인 '**Projection**' 과 '**Manifold Learning**'에 대해 알아볼 것이다

</br>

### 8.2.1 Projection(투영)

: 일반적으로 대부분의 real-world problems 에서 모든 데이터의 features, 즉 dimension이 고르게 분포되어 있지 않다. 

데이터셋인 MNIST를 예로들면, 어떤 특성(각 pixel을 하나의 feature로 볼 때)은 거의 변화가 없고, 또 어떤 feature는 다른 feature들과 서로 연관되어 있다. 

이렇듯 학습 데이터셋은 고차원 공간에서 저차원 부분 공간(subspace)에 위치하게 된다. 

즉, 고차원의 데이터의 feature 중 일부 feature으로 데이터를 표현할 수 있다는 말이 된다.

![img_2](./imgs/8-2-1.PNG)

위의 그림은 왼쪽의 3차원 공간상의 데이터를 2차원 subspace로 projection 시켜, 2차원 dataset으로 만든 것이다.

그러나, projection은 항상 좋은 방법은 아니다. 

</br>

-> 몇몇의 경우에는 subspace가 꼬이거나 뒤집혀 있는데 이런 경우를 ***Swiss roll*** toy dataset 이라 부르며 아래의 그림과 같다.

![img_2](./imgs/8-2-2.PNG)

이러한 유형의 dataset을 그저 평면으로 projecting 시킨다면, 왼쪽 아래와 같이 다른 층들간에 충돌이 일어난다. 

그러나, 우리가 원하는 것은 오른쪽 아래의 그림처럼 datasetdl unroll 되는것을 원한다.

![img_2](./imgs/8-2-3.PNG)

</br>

___

### 8.2.2 Manifold Learning

#### < 정의 >

> ![img_5](./imgs/8-2-4.PNG) </br>
> manifold는 다양체라고도 하며, 국소적으로 유클리드 공간과 닮은 위상 공간이다. </br> 즉, 국소적으로는 유클리드 공간과 구별할 수 없으나, 대역적으로 독특한 위상수학적 구조를 가질 수 있다 (출처 : 위키피디아). </br> 예를 들어, 아래의 원 그림은 모든 점에 대해서 국소적으로 직선과 같은 구조를 갖는 1차원 manifold이다. </br>


</br>

아래의 그림은 **Swis roll** dataset이며, 2D-manifold의 예시로 쓰인다. 

그림에서 볼 수 있듯 2D-manifold는 고차원(3D) 공간에서 휘거나 말린 2D 모양이다. 

일반적으로 d-차원 manifold는 국소적으로 d-차원 초평면으로 볼 수 있는 *n*-차원 공간의 일부이다. (*d < n*)

Swiss roll은 *d* = 2 이고, *n* = 3 인 국소적으로는 2D 평면이지만, 3차원으로 말려있는 dataset이다.

![img_6](./imgs/8-2-5.PNG)

</br>

대부분의 **Dimensionality Reduction** algorithm은 이러한 manifold를 modeling하는 방식으로 동작하며, 이를 **Manifold Learning** 이라고 한다.

이 Manifold Learning은 **Manifold assumption(매니폴드 가정)** 또는 **Manifold hypothesis** 에 의해, 고차원인 실제 dataset이 더 낮은 저차원 manifold에 가깝게 놓여 있다고 가정한다.

**Manifold assumption** 은 종종 다른 가정과 같이 쓰인다.

예를들어, classifier 나 Regression 과 같은 작업을 하기전에 training dataset을 저차원의 manifold 공산으로 표현하면 더 간단하게 문제를 해결 할 수 있다는 가정을 할 수 있다.

![img_7](./imgs/8-2-6.PNG)

</br>

하지만, 위와같은 가정이 항상 통하지 않는다. 

아래의 그림처럼 저차원의 manifold가 오히려 ***Decision Boundary***를 찾는 것이 더 어려운 것을 알 수 있다.

![img_8](./imgs/8-2-7.PNG)

</br>

따라서 model을 학습시키기 전에 training dataset의 차원을 감소시키면, 학습속도는 빨라지지만 model의 성능은 항상 더 나아지거나 더 간단한 model이 되는것은 아니다. 

이는 dataset의 형태에 따라 달라진다.

</br>

___

## 8.3 PCA (Principal Component Analysis)

주성분 분석 (PCA, Principal Component Analysis)는 가장 대표적인 차원축소 algorithm이다.

PCA는 먼저 데이터에 가장 가까운 초평면(hyperplane)을 구한 다음, 데이터를 이 초평면에 projection 시킨다.

</br>



### 8.3.1 Preserving the Variance (분산 보존)

저차원의 hyper-plane에 데이터를 projection하기 이전에 먼저 적절한 hyper-plane을 선택해야한다.

PCA는 데이터의 분상이 최대가 되는 axis를 찾는다. 

즉, 원본 dataset과 projection dataset 간의 **평균제곱거리(MSE)**를 **최소화** 하는 축을 찾는다.

아래의 그림처럼, 왼쪽의 2차원 dataset을 오른쪽 그림처럼 projection 했을 때, C1축으로 투영한 데이터가 분산이 최대로 보존되는 것을 확인 할 수 있다. 

![img_7](./imgs/8-3-1.PNG)

</br>

___

### 8.3.2 Principal Component (주 성분)

PCA는 다음과 같은 단계로 이루어진다.

1. training dataset에서 variance(분산)이 최대인 axis를 찾는다.
2. 이렇게 찾은 첫번째 axis와 othogonal(직교)하면서, variance(분산)이 최대인 두 번째 axis를 찾는다.
3. 첫 번째 axis와 두 번째 axis에 othogonal(직교)하고 variance(분산)을 최대한 보존하는 세 번째 axis를 찾는다.
4. 1~3 과 같은 방법을 연속해서, dataset의 dimension(특성 수) 만큼의 axis를 찾는다.

이렇게 *i*-번째 axis를 정의하는 unit vector(단위 벡터)를 *i*-번째 Principal Component(PC, 주 성분)이라고 한다.

**< *NOTE* >**

> 주요 구성 요소의 방향이 안정적이지 않음: </br>
> Training dataset을 약간 흩트리고 PCA를 다시 실행하면, 새로운 PCs(주 성분들) 중 일부는 원래 PCs와 반대 방향을 가리킬 수 있다.
> 하지만, 그들은 여전히 같은 axis에 누워있을 것이다. </br>
> 어떤 경우에는 PC 한 쌍이 회전하거나 교체될 수도 있지만, 그들이 정의하는 평면은 일반적으로 동일하게 유지된다.

</br>

#### training set 의 principal components (주성분)을 찾는 방법?

: ***Singular Value Decomposition(SVD, 특이값 분해)*** 라 불리는 standard matrix factorization technique(기준 행렬 분해 기술)를 사용하면 된다.

**SVD (특이값 분해)** 는 m x n 행렬에 대해 아래와 같이 정의 된다.

![img_8](./imgs/8-3-2.png)

이는 또한 Python의 Numpy scd() method를 사용하여 PCs(주성분)을 구할 수 있다.

</br>

**< *NOTE* >**

> PCA 는 원점 근처를 기준으로 dataset이 위치해 있음을 가정으로 한다. </br>
> Scikit-Learn 에서의 PCA class 는 data centering을 지원한다. </br>
> 하지만, 나만의 PCA를 설계한다거나 Scikit-Learn 이외의 라이브러리를 사용할 경우, data centering을 먼저 해야한다. 

</br>

___

#### 1) Scikit-Learn 을 통해 PCA 계산

Scikit Learn에서 PCA class를 통해 PCA응 계산할때, dataset에 대한 covariance(공분산)의 고유값 분해(eigen-value decomposition)이 아닌 특이값 분해(SVD, Singular Value Decomposition)를 이용해 계산한다.

#### 2) Example 1-1 : compute PCA


In [26]:
from pprint import pprint
import numpy as np

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5

X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles) / 2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

print('X.shape:', X.shape)

X.shape: (60, 3)


___
#### Example 1-2 : eigen decomposition 

eigen decomposition을 사용하기 위해서는 먼저 covariance (공분산)을 구해야한다.

In [27]:
X_cen = X - X.mean(axis=0) #평균을 0으로
X_cov = np.dot(X_cen.T, X_cen) / 59
print(X_cov)

[[0.69812855 0.17640539 0.12137931]
 [0.17640539 0.1801727  0.07253614]
 [0.12137931 0.07253614 0.04552382]]


</br>
위에서 구한 공분산 행렬을 X_cov에 대해 np.linalg.eig 를 이용해 eigen-value (w) 와 eigen-vector (v) 를 구할 수 있다.

In [30]:
w, v = np.linalg.eig(X_cov)

print("eigen-value : ", w)
print("eigen-vector : ", v)

eigen-value :  [0.77830975 0.1351726  0.01034272]
eigen-vector :  [[ 0.93636116  0.34027485 -0.08626012]
 [ 0.29854881 -0.90119108 -0.31420255]
 [ 0.18465208 -0.2684542   0.94542898]]


In [31]:
print('explained variance ratio : ', w / w.sum())

explained variance ratio :  [0.84248607 0.14631839 0.01119554]


___

</br>

#### Example 1-3 : SVD를 이용한 PCA 구하기

: SVD (특이값 분해)는 covariance(공분산) 행렬 X_cov를 사용하지 않고, X_cen 의 Singular-value 와 Singular-vector를 계산한다.

마찬가지로 np.linalg.svd를 이용하여 PCA를 구할 수 있다.

In [34]:
U, D, V_t = np.linalg.svd(X_cen)

print('Singular value : ', D)
print('Singular vector : \n', V_t.T)

Singular value :  [6.77645005 2.82403671 0.78116597]
Singular vector : 
 [[ 0.93636116 -0.34027485 -0.08626012]
 [ 0.29854881  0.90119108 -0.31420255]
 [ 0.18465208  0.2684542   0.94542898]]


In [35]:
print('explained variance ratio : ', D ** 2 / np.sum(D ** 2))

explained variance ratio :  [0.84248607 0.14631839 0.01119554]


</br

___

#### Example 1-4 : Scikit-Learn을 이용한 PCA 구하기

: Scikit Learn의 PCA를 이용해 위에 과정보다 더 쉽게 구할 수 있다.

+ Scikit-Learn은 편차 또한 자동으로 처리해 계산해 준다.
+ 아래의 example code에서 sigular vector, 즉, PC(주 성분) 행렬을 보면 위의 결과와 부호가 다른 것을 볼 수 있다.
+ 이는 벡터의 방향만 반대일뿐, PC vector가 구성하는 axis는 동일하다.

In [38]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [44]:
print('Singular value : ', pca.singular_values_)
print('Singular vector : \n', pca.components_.T)
print('explained variance ratio : ', pca.explained_variance_ratio_)

Singular value :  [6.77645005 2.82403671 0.78116597]
Singular vector : 
 [[-0.93636116  0.34027485 -0.08626012]
 [-0.29854881 -0.90119108 -0.31420255]
 [-0.18465208 -0.2684542   0.94542898]]
explained variance ratio :  [0.84248607 0.14631839 0.01119554]


< ***NOTE*** >

+ Scikit-Learn 0.18 버전 부터 covariance(공분산) 행렬의 eigen-value 또한 확인 할 수 있다.

In [42]:
print('Singular value : ', pca.explained_variance_)
print('explained variance ratio : ', pca.explained_variance_ratio_)

Singular value :  [0.77830975 0.1351726  0.01034272]
explained variance ratio :  [0.84248607 0.14631839 0.01119554]


</br>

___
### 8.3.3 Explained Variance Ratio와 적절한 차원수 선택하기

: **Explained Variance Ratio**는 각각의 주성분 vector가 이루는 axis에 projection한 결과의 분산의 비율을 말하며,

각 eigen-value의 비율과 같은 의미이다.

아래의 코드는 위의 예제에서 구한 PC-vector를 이용해, projection한 뒤, 분산의 비율(Explained Variance Ratio)을 계산한 코드이다.

In [47]:
#PC1에 projection
pc1 = v[:,0]
proj1 = np.dot(X, pc1)

#PC2에 projection
pc2 = v[:,1]
proj2 = np.dot(X, pc2)

#PC3에 projection
pc3 = v[:,2]
proj3 = np.dot(X, pc3)

proj_list = np.array([proj1.var(), proj2.var(), proj3.var()])

print("Variance (==eigen-value) : ", proj_list)
print('explained variance ratio : ', proj_list / proj_list.sum())

Variance (==eigen-value) :  [0.76533792 0.13291972 0.01017034]
explained variance ratio :  [0.84248607 0.14631839 0.01119554]


</br>

+ Scikit-Learn의 PCA에서는 '**explained_variance_ratio_**'를 통해 위의 코드들을 한 줄로 구할 수 있다.

In [48]:
print("explained variance ratio : ", pca.explained_variance_ratio_)

explained variance ratio :  [0.84248607 0.14631839 0.01119554]


</br>

위의 분석 결과의 의미는 원 dataset 분산의 84.2%가 첫 번째 axis에 놓여 있고, 14.6%가 두 번째 axis에 놓여 있다. 그리고 1.1% 정도의 적은 수의 정보가 세 번째 axis에 놓여 있다.

따라서, 첫 번째 주 성분과 두 번째 주 성분을 이용하여, 3차원의 dataset X 를 2차원으로 projection 할 경우,

원래 dataset의 분산에서 1.1%를 잃게 된다.

</br>

#### **적절한 차원 수 선택하기**

: "**Explained Variance Ratio**"를 이용해 축소할 차원의 수를 고를수 있다.

예를 들면, 누적된 분산의 비율이 95%가 되는 주성분 axis, 차원을 선택하는 것과 같은 방법이다.

In [71]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

print('선택할 차원의 수 :', d)

선택할 차원의 수 : 2


</br>

+ Scikit-Learn의 PCA에서는 n_components에 0 ~ 1사이의 보존하고 싶은 분산의 비율(ratio of variance) 값을 지정해, PCA 계산을 할 수 있다.

+ 차원의 수에 따른 explained variance 변화 함수를 plot 한, 아래의 표 처럼 explained variance 가 급격히 올라가는 elbow 부분 즉, 100차원 정도 밑으로 dimensionality reduction을 진행한다면 explained variance를 많이 잃지 않을 것이다. 

![img_11](./imgs/8-3-6.PNG)

In [77]:
pca = PCA(n_components=.95)
X_proj = pca.fit_transform(X)

print('Principal Component Vector : \n', pca.components_.T)

Principal Component Vector : 
 [[-0.93636116  0.34027485]
 [-0.29854881 -0.90119108]
 [-0.18465208 -0.2684542 ]]


</br>

___

### 8.3.4 압축을 위한 PCA 

### < Reference >

책 : Hands on Machine Learning

code 및 images : https://excelsior-cjh.tistory.com/167?category=918734 [EXCELSIOR]